In [1]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [12]:
#here Zwe will try to perform following task:
# reading from a csv file 
#Performing Transformation using Dataframes
# RDD transformations

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

#Step 1 Create a Spark Object 
#Step 2: Loading CSV file
#Step 3: Creating Dataframe & Displaying its Content 
#Step 4: Performing transformations 
#Step 5: Closing spark session

# Create a Spark session
spark = SparkSession.builder \
    .appName("Demo 2: PySpark CSV and RDD Demo") \
    .getOrCreate()

#df = spark.read.csv('/Dataset/employee.csv', header= True, inferSchema=True)

# Define Schema
schema = StructType([
    StructField("ID", IntegerType(), True),
    StructField("Name", StringType(), True),
    StructField("Age", IntegerType(), True),
    StructField("Role", StringType(), True),
    StructField("Salary", IntegerType(), True)
])

# Create Data
data = [
    (1, "Alice", 25, "Data Analyst", 60000),
    (2, "Bob", 30, "Software Engineer", 80000),
    (3, "Charlie", 28, "Data Scientist", 90000),
    (4, "David", 35, "Manager", 120000),
    (5, "Eva", 32, "Software Engineer", 85000)
]

#creating Dataframe 
df = spark.createDataFrame(data,schema=schema)

#displaying Dataframe
df.show()


+---+-------+---+-----------------+------+
| ID|   Name|Age|             Role|Salary|
+---+-------+---+-----------------+------+
|  1|  Alice| 25|     Data Analyst| 60000|
|  2|    Bob| 30|Software Engineer| 80000|
|  3|Charlie| 28|   Data Scientist| 90000|
|  4|  David| 35|          Manager|120000|
|  5|    Eva| 32|Software Engineer| 85000|
+---+-------+---+-----------------+------+



In [13]:
#filter Emp with salary >80,0000 INR

df.filter(col("salary")>80000).show()

+---+-------+---+-----------------+------+
| ID|   Name|Age|             Role|Salary|
+---+-------+---+-----------------+------+
|  3|Charlie| 28|   Data Scientist| 90000|
|  4|  David| 35|          Manager|120000|
|  5|    Eva| 32|Software Engineer| 85000|
+---+-------+---+-----------------+------+



In [15]:
df.groupBy("Role").agg(avg("Salary").alias("AverageSalary")).show()

+-----------------+-------------+
|             Role|AverageSalary|
+-----------------+-------------+
|   Data Scientist|      90000.0|
|     Data Analyst|      60000.0|
|          Manager|     120000.0|
|Software Engineer|      82500.0|
+-----------------+-------------+



In [17]:
#For COnverting Dataframe to RDD(Resilient Distributed Datset)
#It reprsents an inmutable, partitioned collection of elements that can be operated on in prallel
rdd = df.rdd

#Show first few records
df.show()
for row in rdd.take(3):
    print(row)

+---+-------+---+-----------------+------+
| ID|   Name|Age|             Role|Salary|
+---+-------+---+-----------------+------+
|  1|  Alice| 25|     Data Analyst| 60000|
|  2|    Bob| 30|Software Engineer| 80000|
|  3|Charlie| 28|   Data Scientist| 90000|
|  4|  David| 35|          Manager|120000|
|  5|    Eva| 32|Software Engineer| 85000|
+---+-------+---+-----------------+------+

Row(ID=1, Name='Alice', Age=25, Role='Data Analyst', Salary=60000)
Row(ID=2, Name='Bob', Age=30, Role='Software Engineer', Salary=80000)
Row(ID=3, Name='Charlie', Age=28, Role='Data Scientist', Salary=90000)


In [19]:
#Map transforamtion( Extract name and Salries)

name_salary_rdd = rdd.map(lambda x: (x["Name"], x["Salary"]))
print(name_salary_rdd.collect())


[('Alice', 60000), ('Bob', 80000), ('Charlie', 90000), ('David', 120000), ('Eva', 85000)]
[Row(ID=4, Name='David', Age=35, Role='Manager', Salary=120000), Row(ID=5, Name='Eva', Age=32, Role='Software Engineer', Salary=85000)]


In [20]:
#Filter Transforamtion(Emp with Age > 30)
filtered_rdd = rdd.filter(lambda x: x["Age"] > 30)
print(filtered_rdd.collect())


[Row(ID=4, Name='David', Age=35, Role='Manager', Salary=120000), Row(ID=5, Name='Eva', Age=32, Role='Software Engineer', Salary=85000)]


In [21]:
#reduce by key( Total salary by role)
role_salary_rdd = rdd.map(lambda x: (x["Role"], x["Salary"])).reduceByKey(lambda x, y: x + y)
print(role_salary_rdd.collect())


[('Data Analyst', 60000), ('Software Engineer', 165000), ('Manager', 120000), ('Data Scientist', 90000)]


In [22]:
spark.stop()